<a href="https://colab.research.google.com/github/pcamarillor/O2024_ESI3914O/blob/pablo_camarillo_lecture_10/examples/notebooks/columns_operations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!sudo apt update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
# Check this site for the latest download link
# https://www.apache.org/dyn/closer.lua/spark
!wget -q https://dlcdn.apache.org/spark/spark-3.5.2/spark-3.5.2-bin-hadoop3.tgz
!tar xf spark-3.5.2-bin-hadoop3.tgz
!pip install -q findspark
!pip install pyspark
!pip install py4j

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Ign:3 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:5 https://r2u.stat.illinois.edu/ubuntu jammy Release [5,713 B]
Hit:6 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy Release.gpg [793 B]
Get:8 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:9 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,149 kB]
Get:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Hit:11 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:12 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:13 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [2,224 kB]
Hi

In [21]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, FloatType
from pyspark.sql import Row
from pyspark.sql.functions import to_date, when, col

# Initialize Spark session
spark = SparkSession.builder \
    .appName("E-Commerce App DataFrame with Mixed Date Formats") \
    .getOrCreate()

# Define the schema (excluding order_date, it will be derived from order_date_str)
schema = StructType([
    StructField("order_id", IntegerType(), nullable=False),
    StructField("product", StringType(), nullable=False),
    StructField("price", FloatType(), nullable=False),
    StructField("quantity", IntegerType(), nullable=False),
    StructField("discount", FloatType(), nullable=True),
    StructField("first_name", StringType(), nullable=False),
    StructField("last_name", StringType(), nullable=False),
    StructField("order_date_str", StringType(), nullable=False)  # String date with mixed formats
])

# Sample data with mixed date formats in 'order_date_str'
data = [
    Row(order_id=1, product="Laptop", price=1200.00, quantity=1, discount=100.00, first_name="John", last_name="Doe", order_date_str="2024-09-01"),
    Row(order_id=2, product="Smartphone", price=800.00, quantity=2, discount=50.00, first_name="Jane", last_name="Smith", order_date_str="09/02/2024"),
    Row(order_id=3, product="Headphones", price=200.00, quantity=3, discount=20.00, first_name="Alice", last_name="Brown", order_date_str="2024-09-03"),
    Row(order_id=4, product="Monitor", price=300.00, quantity=2, discount=30.00, first_name="Bob", last_name="Johnson", order_date_str="09/04/2024"),
    Row(order_id=5, product="Keyboard", price=100.00, quantity=5, discount=10.00, first_name="Eve", last_name="Williams", order_date_str="2024-09-05")
]

# Create the DataFrame
df = spark.createDataFrame(data, schema)

# Show the updated DataFrame with the correct 'order_date' column
df.show()

# Print the schema to verify the new 'order_date' column is of DateType
df.printSchema()


+--------+----------+------+--------+--------+----------+---------+--------------+
|order_id|   product| price|quantity|discount|first_name|last_name|order_date_str|
+--------+----------+------+--------+--------+----------+---------+--------------+
|       1|    Laptop|1200.0|       1|   100.0|      John|      Doe|    2024-09-01|
|       2|Smartphone| 800.0|       2|    50.0|      Jane|    Smith|    09/02/2024|
|       3|Headphones| 200.0|       3|    20.0|     Alice|    Brown|    2024-09-03|
|       4|   Monitor| 300.0|       2|    30.0|       Bob|  Johnson|    09/04/2024|
|       5|  Keyboard| 100.0|       5|    10.0|       Eve| Williams|    2024-09-05|
+--------+----------+------+--------+--------+----------+---------+--------------+

root
 |-- order_id: integer (nullable = false)
 |-- product: string (nullable = false)
 |-- price: float (nullable = false)
 |-- quantity: integer (nullable = false)
 |-- discount: float (nullable = true)
 |-- first_name: string (nullable = false)
 |--

In [22]:
from pyspark.sql.functions import lit
df = df.withColumn("new_column", lit(10))
df.show()

+--------+----------+------+--------+--------+----------+---------+--------------+----------+
|order_id|   product| price|quantity|discount|first_name|last_name|order_date_str|new_column|
+--------+----------+------+--------+--------+----------+---------+--------------+----------+
|       1|    Laptop|1200.0|       1|   100.0|      John|      Doe|    2024-09-01|        10|
|       2|Smartphone| 800.0|       2|    50.0|      Jane|    Smith|    09/02/2024|        10|
|       3|Headphones| 200.0|       3|    20.0|     Alice|    Brown|    2024-09-03|        10|
|       4|   Monitor| 300.0|       2|    30.0|       Bob|  Johnson|    09/04/2024|        10|
|       5|  Keyboard| 100.0|       5|    10.0|       Eve| Williams|    2024-09-05|        10|
+--------+----------+------+--------+--------+----------+---------+--------------+----------+



In [23]:
df = df.drop("new_column")
df = df.withColumn('total_cost', \
      df['price'] * df['quantity'])
df.show()

+--------+----------+------+--------+--------+----------+---------+--------------+----------+
|order_id|   product| price|quantity|discount|first_name|last_name|order_date_str|total_cost|
+--------+----------+------+--------+--------+----------+---------+--------------+----------+
|       1|    Laptop|1200.0|       1|   100.0|      John|      Doe|    2024-09-01|    1200.0|
|       2|Smartphone| 800.0|       2|    50.0|      Jane|    Smith|    09/02/2024|    1600.0|
|       3|Headphones| 200.0|       3|    20.0|     Alice|    Brown|    2024-09-03|     600.0|
|       4|   Monitor| 300.0|       2|    30.0|       Bob|  Johnson|    09/04/2024|     600.0|
|       5|  Keyboard| 100.0|       5|    10.0|       Eve| Williams|    2024-09-05|     500.0|
+--------+----------+------+--------+--------+----------+---------+--------------+----------+



In [24]:
df = df.withColumn("final_price", df['total_cost'] - df['discount'])
df.show()

+--------+----------+------+--------+--------+----------+---------+--------------+----------+-----------+
|order_id|   product| price|quantity|discount|first_name|last_name|order_date_str|total_cost|final_price|
+--------+----------+------+--------+--------+----------+---------+--------------+----------+-----------+
|       1|    Laptop|1200.0|       1|   100.0|      John|      Doe|    2024-09-01|    1200.0|     1100.0|
|       2|Smartphone| 800.0|       2|    50.0|      Jane|    Smith|    09/02/2024|    1600.0|     1550.0|
|       3|Headphones| 200.0|       3|    20.0|     Alice|    Brown|    2024-09-03|     600.0|      580.0|
|       4|   Monitor| 300.0|       2|    30.0|       Bob|  Johnson|    09/04/2024|     600.0|      570.0|
|       5|  Keyboard| 100.0|       5|    10.0|       Eve| Williams|    2024-09-05|     500.0|      490.0|
+--------+----------+------+--------+--------+----------+---------+--------------+----------+-----------+



In [25]:
from pyspark.sql.functions import when
df = df.withColumn('is_high_value', when(df['final_price'] > 1000, "YES").otherwise("NO"))

In [26]:
df.show()

+--------+----------+------+--------+--------+----------+---------+--------------+----------+-----------+-------------+
|order_id|   product| price|quantity|discount|first_name|last_name|order_date_str|total_cost|final_price|is_high_value|
+--------+----------+------+--------+--------+----------+---------+--------------+----------+-----------+-------------+
|       1|    Laptop|1200.0|       1|   100.0|      John|      Doe|    2024-09-01|    1200.0|     1100.0|          YES|
|       2|Smartphone| 800.0|       2|    50.0|      Jane|    Smith|    09/02/2024|    1600.0|     1550.0|          YES|
|       3|Headphones| 200.0|       3|    20.0|     Alice|    Brown|    2024-09-03|     600.0|      580.0|           NO|
|       4|   Monitor| 300.0|       2|    30.0|       Bob|  Johnson|    09/04/2024|     600.0|      570.0|           NO|
|       5|  Keyboard| 100.0|       5|    10.0|       Eve| Williams|    2024-09-05|     500.0|      490.0|           NO|
+--------+----------+------+--------+---

In [27]:
from pyspark.sql.functions import concat

df = df.withColumn('full_name', concat(df['first_name'], lit(' '), df['last_name']))
df_reduced = df.select("first_name", "last_name", "full_name")
df_reduced.show()

+----------+---------+------------+
|first_name|last_name|   full_name|
+----------+---------+------------+
|      John|      Doe|    John Doe|
|      Jane|    Smith|  Jane Smith|
|     Alice|    Brown| Alice Brown|
|       Bob|  Johnson| Bob Johnson|
|       Eve| Williams|Eve Williams|
+----------+---------+------------+



In [31]:
df = df.withColumn('order_date',
    when(col('order_date_str').rlike("^\d{4}-\d{2}-\d{2}$"), to_date(col('order_date_str'), 'yyyy-MM-dd'))
    .when(col('order_date_str').rlike("^\d{2}/\d{2}/\d{4}$"), to_date(col('order_date_str'), 'MM/dd/yyyy'))
    .otherwise(None)  # Handle unexpected formats with None
)
df.select("order_id", "product", "order_date_str", "order_date").show()

+--------+----------+--------------+----------+
|order_id|   product|order_date_str|order_date|
+--------+----------+--------------+----------+
|       1|    Laptop|    2024-09-01|2024-09-01|
|       2|Smartphone|    09/02/2024|2024-09-02|
|       3|Headphones|    2024-09-03|2024-09-03|
|       4|   Monitor|    09/04/2024|2024-09-04|
|       5|  Keyboard|    2024-09-05|2024-09-05|
+--------+----------+--------------+----------+



In [32]:
from pyspark.sql.functions import year, month

df = df.withColumn('order_year', year(df['order_date']))
df = df.withColumn('order_month', month(df['order_date']))

In [33]:
df.select("order_id", "product", "order_year", "order_month").show()

+--------+----------+----------+-----------+
|order_id|   product|order_year|order_month|
+--------+----------+----------+-----------+
|       1|    Laptop|      2024|          9|
|       2|Smartphone|      2024|          9|
|       3|Headphones|      2024|          9|
|       4|   Monitor|      2024|          9|
|       5|  Keyboard|      2024|          9|
+--------+----------+----------+-----------+



In [34]:
df = df.withColumn('price_int', df['price'].cast('integer'))
df.select("order_id", "price", "price_int").show()

+--------+------+---------+
|order_id| price|price_int|
+--------+------+---------+
|       1|1200.0|     1200|
|       2| 800.0|      800|
|       3| 200.0|      200|
|       4| 300.0|      300|
|       5| 100.0|      100|
+--------+------+---------+



In [35]:
df = df.withColumnRenamed('price_int', 'price_integer')
df.select("order_id", "price", "price_integer").show()

+--------+------+-------------+
|order_id| price|price_integer|
+--------+------+-------------+
|       1|1200.0|         1200|
|       2| 800.0|          800|
|       3| 200.0|          200|
|       4| 300.0|          300|
|       5| 100.0|          100|
+--------+------+-------------+



In [ ]:
spark.stop()